Bu proje, Boston'daki gayrimenkul fiyatlarını tahmin etmek amacıyla regresyon ağaçları (regression trees) kullanarak bir model oluşturmayı hedeflemektedir. Veriler, çeşitli bölgelerdeki evlerin ortalama fiyatını etkileyen faktörleri içermektedir. Bu projede, regresyon ağaçları kullanarak modelin nasıl eğitileceği, performansının nasıl değerlendirileceği ve tahmin hatalarının nasıl hesaplanacağı incelenmektedir. Sonuç olarak, modelin doğru fiyat tahminleri yapıp yapmadığını belirlemek için R² değeri ve ortalama hata gibi metrikler kullanılacaktır.

# Veriyi Okuma


In [66]:
import pandas as pd

data = pd.read_csv('real_estate_data.csv')

data.head()

,tx_price,beds,baths,sqft,year_built,lot_size,property_type,exterior_walls,roof,basement,...,beauty_spas,active_life,median_age,married,college_grad,property_tax,insurance,median_school,num_schools,tx_year
0,295850,1,1,584,2013,0,Apartment / Condo / Townhouse,Wood Siding,NaN,NaN,...,47,58,33.0,65.0,84.0,234.0,81.0,9.0,3.0,2013
1,216500,1,1,612,1965,0,Apartment / Condo / Townhouse,Brick,Composition Shingle,1.0,...,26,14,39.0,73.0,69.0,169.0,51.0,3.0,3.0,2006
2,279900,1,1,615,1963,0,Apartment / Condo / Townhouse,Wood Siding,NaN,NaN,...,74,62,28.0,15.0,86.0,216.0,74.0,8.0,3.0,2012
3,379900,1,1,618,2000,33541,Apartment / Condo / Townhouse,Wood Siding,NaN,NaN,...,72,83,36.0,25.0,91.0,265.0,92.0,9.0,3.0,2005
4,340000,1,1,634,1992,0,Apartment / Condo / Townhouse,Brick,NaN,NaN,...,50,73,37.0,20.0,75.0,88.0,30.0,9.0,3.0,2002


# Veri Boyutunu İnceleme

In [67]:
data.shape


(1883, 26)

# Eksik Değerlerin Kontrolü

In [68]:
data.isna().sum()


tx_price                0
beds                    0
baths                   0
sqft                    0
year_built              0
lot_size                0
property_type           0
exterior_walls        223
roof                  354
basement              226
restaurants             0
groceries               0
nightlife               0
cafes                   0
shopping                0
arts_entertainment      0
beauty_spas             0
active_life             0
median_age              0
married                 0
college_grad            0
property_tax            0
insurance               0
median_school           0
num_schools             0
tx_year                 0
dtype: int64

# Veri Ön İşleme

In [69]:
data.dropna(inplace=True)
data.isna().sum()


tx_price              0
beds                  0
baths                 0
sqft                  0
year_built            0
lot_size              0
property_type         0
exterior_walls        0
roof                  0
basement              0
restaurants           0
groceries             0
nightlife             0
cafes                 0
shopping              0
arts_entertainment    0
beauty_spas           0
active_life           0
median_age            0
married               0
college_grad          0
property_tax          0
insurance             0
median_school         0
num_schools           0
tx_year               0
dtype: int64

#  Özellik ve Hedef Değişkenlerin Belirlenmesi

In [70]:
X = data.drop(columns=["tx_price"])
Y = data["tx_price"]


# Eğitim ve Test Verisine Ayırma

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)


# Regresyon Ağacının Oluşturulması

In [72]:
from sklearn.tree import DecisionTreeRegressor
regression_tree = DecisionTreeRegressor(criterion="squared_error")


#  Modelin Eğitilmesi

In [73]:
X_train = pd.get_dummies(X_train, drop_first=True) #X_train veri setindeki kategorik değişkenler, sayısal değerlere dönüştürülüyor
X_test = pd.get_dummies(X_test, drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

regression_tree.fit(X_train, Y_train)


DecisionTreeRegressor()

# Modelin Değerlendirilmesi

In [74]:
score = regression_tree.score(X_test, Y_test)
print(score)

0.584942253801995


Bu sonuc, modelin bazı ölçüde etkili olduğunu, ancak iyileştirmeye ihtiyaç duyduğunu gösterir.

Geliştirme Olanakları: Modelin performansını artırmak için:

Daha fazla özellik eklemeyi düşünebilirsiniz.
Model parametrelerini optimize edebilir (örneğin, derinliği ayarlayarak).
Farklı bir model (örneğin, rastgele orman, destek vektör makineleri) deneyebilirsiniz.

# Ortalama Hatanın Hesaplanması

In [76]:
import numpy as np

# Tahminleri yap
prediction = regression_tree.predict(X_test)

# Ortalama hatayı hesapla
average_error = np.abs(prediction - Y_test.values).mean() * 1000  # NumPy kullan
print("$", average_error)

$ 70120368.21705426


Tahmin: regression_tree.predict(X_test) ile modelin test verisi üzerindeki tahminleri alınır.

Hata Hesaplama: (prediction - Y_test).abs().mean() ifadesi, tahmin ile gerçek değerler arasındaki mutlak farkların ortalamasını hesaplar. Bu, ortalama mutlak hatayı (MAE) verir.

Büyütme: Sonuç, 1000 ile çarpılır. Bu, hata değerini daha anlamlı bir ölçekte (örneğin, bin dolara kadar) ifade etmek içindir.

Sonuç: Son olarak, hesaplanan hata değeri ekrana yazdırılır.

70334290.70 dolarlık bir ortalama hata oldukça yüksek. Bu, modelin test verisi üzerindeki tahminlerinin genellikle gerçek değerlere göre büyük sapmalar gösterdiğini gösteriyor. Bu durumda, modelin performansını artırmak için özellik mühendisliği, daha uygun bir model seçimi veya hiperparametre ayarlamaları gibi adımları değerlendirmek faydalı olabilir. Ayrıca, verilerin kalitesini ve dağılımını incelemek de önemli olabilir.

# Alternatif Kriter ile Model Eğitimi

In [56]:
regression_tree = DecisionTreeRegressor(criterion="absolute_error")
regression_tree.fit(X_train, Y_train)
print(regression_tree.score(X_test, Y_test))
prediction = regression_tree.predict(X_test)
print("$", (prediction - Y_test).abs().mean() * 1000)

0.526655008650937
$ 76293379.84496124
